# PPA Parking Violations (2017) Data Analysis

This data analysis will include SQL, Python, and data visualization

## Table of Contents:
* [Setup](#setup)
    * [Importing libraries](#import-libraries)
    * [Importing data](#import-data)
    * [Creating sqlite server](#create-sqlite-server)
* [Analysis](#analysis)
    * [How many tickets were given in 2017?](#total-tickets-2017)
    * [How much fines did the PPA give in 2017?](#ppa-fines-2017)
    * [Which agency gave out the most tickets?](#agency-tickets-given)
    * [What are the revenue driver for the PPA?](#ppa-revenue-drivers)
    * [What are the top five states that gets fined?](#top-five-states-fined)
    * [What is the distrubtion of tickets given throughout the day?](#distribution-day)
    * [What is the distrubtion of tickets given throughout the week?](#distribution-week)
    * [What is the distrubtion of tickets given throughout the year?](#distribution-year)
    * [Do certain zipcodes get more fines than other zip codes?](#zipcode-fine-comparison)

## Setup <a class="anchor" id="setup"></a>

### Importing libraries <a class="anchor" id="import-libraries"></a>

We will be importing the following libraries:
1. pandas
    1. Data processing and reading/writing csv files
2. plotly.express
    1. Data visualization
3. sqlite3
    1. SQL querying and creating a quick database

In [23]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import sqlite3

### Importing data <a class="anchor" id="import-data"></a>

In [24]:
PPA_Dataset_1 = pd.read_csv('parking_violations_2017_1.csv')
PPA_Dataset_2 = pd.read_csv('parking_violations_2017_2.csv')
# combine both datasets
PPA_Full_Dataset = pd.concat([PPA_Dataset_1,PPA_Dataset_2])

# change null to 0 and int to get rid of float 19192.0 -> 19192
PPA_Full_Dataset['zip_code'] = PPA_Full_Dataset['zip_code'].fillna(0).astype("int64")

### Creating sqlite server <a class="anchor" id="create-sqlite-server"></a>

In [25]:
# Import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# Connect dataframes to SQL database
PPA_Full_Dataset.to_sql("PPA_2017_Dataset", con=engine)

## Run query function for SQL interaction
def run_query(a):
    df_sql = pd.read_sql_query(a,con=engine)
    return df_sql

## Analysis <a class="anchor" id="analysis"></a>

### How many tickets were given in 2017? <a class="anchor" id="total-tickets-2017"></a>

In [26]:
Total_Tickets = """
SELECT CAST(COUNT(anon_ticket_number) AS int) AS Total_Tickets
FROM PPA_2017_Dataset
""" 
run_query(Total_Tickets)

,Total_Tickets
0,1722620


### Which agency gave out the most tickets? <a class="anchor" id="agency-tickets-given"></a>

In [27]:
TicketsByAgency = """
WITH Total_Tickets AS (
    SELECT CAST(COUNT(anon_ticket_number) AS int) AS Total_Tickets
    FROM PPA_2017_Dataset
)

SELECT 
    issuing_agency AS Agency,
    COUNT(anon_ticket_number) AS Tickets_Given,
    ROUND((CAST(COUNT(anon_ticket_number) AS float) / Total_Tickets) * 100, 2) || '%' AS Percent_Total_Tickets
FROM PPA_2017_Dataset, Total_Tickets
GROUP BY Agency
ORDER BY Tickets_Given DESC
"""
run_query(TicketsByAgency)

,Agency,Tickets_Given,Percent_Total_Tickets
0,PPA,1527876,88.69%
1,POLICE,186304,10.82%
2,CENTER C,2846,0.17%
3,PENN,1407,0.08%
4,SEPTA,1190,0.07%
5,TEMPLE,1040,0.06%
6,HOUSING,1032,0.06%
7,FAIRMNT,465,0.03%
8,PRISONS,434,0.03%
9,POST OFF,16,0.0%


In [28]:
fig = px.bar(run_query(TicketsByAgency), x='Agency', y='Tickets_Given')
fig.show()

### How much fines did the PPA give in 2017? <a class="anchor" id="ppa-fines-2017"></a>

In [29]:
PPAFines2017 = """
SELECT SUM(fine) AS PPA_Total_Fines
FROM PPA_2017_Dataset
WHERE issuing_agency = 'PPA'
"""
run_query(PPAFines2017)

,PPA_Total_Fines
0,67958756


In [30]:
PPAFines2017 = """
WITH Total_Fines_All_Agencies AS (
    SELECT SUM(fine) AS Total_Fines
    FROM PPA_2017_Dataset
)

SELECT issuing_agency, SUM(fine) AS Sum_Fines,
    ROUND((CAST(SUM(fine) AS float) / Total_Fines) * 100, 2) || '%' AS Percent_Total_Fines
FROM PPA_2017_Dataset, Total_Fines_All_Agencies
GROUP by issuing_agency
ORDER BY Sum_Fines DESC
"""
run_query(PPAFines2017)

,issuing_agency,Sum_Fines,Percent_Total_Fines
0,PPA,67958756,83.66%
1,POLICE,12733945,15.68%
2,CENTER C,204681,0.25%
3,PENN,84527,0.1%
4,HOUSING,83882,0.1%
5,SEPTA,69555,0.09%
6,TEMPLE,52810,0.07%
7,FAIRMNT,21355,0.03%
8,PRISONS,19884,0.02%
9,POST OFF,1031,0.0%


### What are the revenue driver for the PPA? <a class="anchor" id="ppa-revenue-drivers"></a>

In [31]:
PPARevenueDrivers = """
WITH TotalRevenue AS (
    SELECT SUM(fine) AS PPA_Total_Revenue
    FROM PPA_2017_Dataset
    WHERE issuing_agency = 'PPA'
)

SELECT 
    TRIM(violation_desc, ' CC') AS ViolationDescription,
    SUM(fine) as Total_Fines,
    ROUND((SUM(fine)/CAST(PPA_Total_Revenue AS FLOAT)) * 100, 2) || '%' AS Percent_Total_Revenue
FROM PPA_2017_Dataset, TotalRevenue
WHERE issuing_agency = 'PPA'
GROUP BY ViolationDescription
ORDER BY Total_Fines DESC
LIMIT 10
"""
run_query(PPARevenueDrivers)

,ViolationDescription,Total_Fines,Percent_Total_Revenue
0,METER EXPIRED,20647426,30.38%
1,STOP PROHIBITED,12342882,18.16%
2,OVER TIME LIMIT,6748822,9.93%
3,HP RESERVED SPACE,5717194,8.41%
4,EXPIRED INSPECTION,5155668,7.59%
5,PARKING PROHBITED,4361765,6.42%
6,STOPPING PROHIBITED,2719065,4.0%
7,BUS ONLY ZONE,2142168,3.15%
8,DOUBLE PARKED,1093191,1.61%
9,LOADING ZONE,1063497,1.56%


In [32]:
fig = px.bar(run_query(PPARevenueDrivers), x='Total_Fines', y='ViolationDescription', orientation='h')
fig.show()

### What are the top five states that gets fined? <a class="anchor" id="top-five-states-fined"></a>

In [33]:
ticketsPerState = """
WITH TotalTickets AS (
    SELECT COUNT(state) AS Total_Tickets
    FROM PPA_2017_Dataset
)

SELECT 
    state, 
    COUNT(state) as Tickets_Given, 
    ROUND(CAST(COUNT(state) AS INT)/CAST(TotalTickets.Total_Tickets AS FLOAT) * 100, 2) || '%' AS Percentage_Of_Tickets
FROM PPA_2017_Dataset, TotalTickets
GROUP BY State 
ORDER BY Tickets_Given DESC
LIMIT 5
"""
run_query(ticketsPerState)

,state,Tickets_Given,Percentage_Of_Tickets
0,PA,1315711,76.38%
1,NJ,189102,10.98%
2,NY,32953,1.91%
3,DE,31481,1.83%
4,MD,22311,1.3%


In [34]:
fig = px.bar(run_query(ticketsPerState), x='state', y='Tickets_Given')
fig.show()

### What is the distrubtion of tickets given throughout the day? <a class="anchor" id="distribution-day"></a>

In [35]:
# playing with dates, check month, day of week, and time for patterns
# https://philapark.org/laws-enforcement/ - does it match the hours?
#6AM to 10:30PM Monday – Wednesday
#6AM to 12AM Thursday
#6AM to 3AM Friday
#6:30AM to 3AM Saturday
#7AM to 11PM Sunday

FinesByHour = """
SELECT
    issue_datetime,
    issuing_agency,
    SUBSTR(issue_datetime, 12, 2) AS Hour, -- Get the hour 2017-02-11 [00]:00:00
    COUNT(anon_ticket_number) AS Total_Tickets
FROM PPA_2017_Dataset
GROUP BY issuing_agency, Hour
ORDER BY Hour
"""
run_query(FinesByHour)

,issue_datetime,issuing_agency,Hour,Total_Tickets
0,2017-02-11 00:00:00,None,00,1
1,2017-03-28 00:00:00,CENTER C,00,21
2,2017-01-16 00:03:00,FAIRMNT,00,3
3,2017-01-15 00:21:00,HOUSING,00,48
4,2017-02-18 00:45:00,PENN,00,53
...,...,...,...,...
216,2017-01-21 23:30:00,PENN,23,20
217,2017-02-26 23:45:00,POLICE,23,6558
218,2017-02-25 23:13:00,PPA,23,10009
219,2017-03-08 23:12:00,SEPTA,23,14


In [36]:
fig = px.histogram(run_query(FinesByHour), x="Hour", y="Total_Tickets", color="issuing_agency")

fig.update_layout(
    bargap=0.0,# gap between bars of adjacent location coordinates
    barmode='stack'
)
fig.show()

### What is the distrubtion of tickets given throughout the week? <a class="anchor" id="distribution-week"></a>

In [37]:
# Fine seperated by day of week
FinesByDayOfWeek = """
WITH FinesByDay AS (
    SELECT
        CASE 
          CAST (strftime('%w', issue_datetime) AS INTEGER)
          WHEN 0 THEN 'Sunday'
          WHEN 1 THEN 'Monday'
          WHEN 2 THEN 'Tuesday'
          WHEN 3 THEN 'Wednesday'
          WHEN 4 THEN 'Thursday'
          WHEN 5 THEN 'Friday'
          ELSE 'Saturday' 
        END AS DayOfWeek,
        CAST (strftime('%w', issue_datetime) AS INTEGER) AS DayOfWeekINT,
        TRIM(violation_desc, ' CC') AS ViolationDescription,
        COUNT(issue_datetime) as CountOfViolations
    FROM PPA_2017_Dataset
    GROUP BY DayOfWeek, ViolationDescription
    ORDER BY DayOfWeekINT
)

SELECT *
FROM FinesByDay
WHERE CountOfViolations > 5000
"""
run_query(FinesByDayOfWeek)

,DayOfWeek,DayOfWeekINT,ViolationDescription,CountOfViolations
0,Sunday,0,EXPIRED INSPECTION,8276
1,Sunday,0,METER EXPIRED,29779
2,Sunday,0,OVER TIME LIMIT,5758
3,Sunday,0,PARKING PROHBITED,9363
4,Sunday,0,STOP PROHIBITED,6057
5,Monday,1,BUS ONLY ZONE,5190
6,Monday,1,EXPIRED INSPECTION,21436
7,Monday,1,METER EXPIRED,85595
8,Monday,1,OVER TIME LIMIT,32049
9,Monday,1,PARKING PROHBITED,19673


In [38]:
fig = px.line(run_query(FinesByDayOfWeek), x="DayOfWeek", y="CountOfViolations", color="ViolationDescription", title="Violations by Day of the Week")
fig.show()

### What is the distrubtion of tickets given throughout the year? <a class="anchor" id="distribution-year"></a>

In [39]:
# Fine seperated by month
FinesByMonth = """
SELECT
    issue_datetime,
    CASE 
        strftime('%m', issue_datetime)
        WHEN '01' THEN 'Jan'
        WHEN '02' THEN 'Feb'
        WHEN '03' THEN 'Mar'
        WHEN '04' THEN 'Apr'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'Jun'
        WHEN '07' THEN 'Jul'
        WHEN '08' THEN 'Aug'
        WHEN '09' THEN 'Sep'
        WHEN '10' THEN 'Oct'
        WHEN '11' THEN 'Nov'
        ELSE 'Dec' 
    END AS Month,
    CAST(strftime('%m', issue_datetime) AS INTEGER) AS MonthINT,
    COUNT(issue_datetime) AS CountOfViolations,
    TRIM(violation_desc, ' CC') AS ViolationDescription
FROM PPA_2017_Dataset
GROUP BY Month, ViolationDescription
ORDER BY MonthINT
"""
run_query(FinesByMonth)

,issue_datetime,Month,MonthINT,CountOfViolations,ViolationDescription
0,2017-01-27 21:47:00,Jan,1,16,AR SHARE VEHS ONLY
1,2017-01-03 15:52:00,Jan,1,806,BLOCKING DRIVEWAY
2,2017-01-24 13:27:00,Jan,1,38,BLOCKNG MASS TRANSIT
3,2017-01-14 21:37:00,Jan,1,6,BUS NOT IN BUS STAND
4,2017-01-06 09:51:00,Jan,1,2985,BUS ONLY ZONE
...,...,...,...,...,...
773,2017-12-12 08:20:00,Dec,12,28,UNATTENDED VEH 29
774,2017-12-19 16:45:00,Dec,12,105,UNREG/ABANDONED VEH
775,2017-12-02 00:42:00,Dec,12,26,VALET VIOLATION
776,2017-12-02 00:58:00,Dec,12,158,VALET ZONE VIOLATION


In [40]:
fig = px.line(run_query(FinesByMonth), x="Month", y="CountOfViolations", color="ViolationDescription", title="Violations by Month")
fig.show()


In [41]:
fig = px.bar(run_query(FinesByMonth), x="Month", y="CountOfViolations", color="ViolationDescription", title="Violations by Month")
fig.show()

### Do certain zipcodes get more fines than other zip codes? <a class="anchor" id="zipcode-fine-comparison"></a> 

In [42]:
FinesPerZipCode = """
SELECT 
    zip_code, 
    COUNT(anon_ticket_number) AS fines_given,
    
    SUM(fine) AS sum_fines
FROM PPA_2017_Dataset
WHERE zip_code != "0"
GROUP BY zip_code 
ORDER BY fines_given DESC
"""
run_query(FinesPerZipCode)

,zip_code,fines_given,sum_fines
0,19103,178096,9719831
1,19107,162393,9086563
2,19147,150897,6437837
3,19104,139237,6323122
4,19106,108665,5777605
5,19146,71698,2869658
6,19130,67078,2793793
7,19102,51456,3026116
8,19148,49532,1915687
9,19123,47373,1997463


In [47]:
# TO DO:
# COMPARE WITH WEATHER! DOES temperature affect amount of tickets (do a scatter 365 days)
# Find geoJSON to create a data visualization showing violations per zipcode.

In [46]:
ViolationByZipCodeRanked = """
WITH TempTable AS (
    SELECT
        zip_code,
        TRIM(violation_desc, ' CC') AS Violation_Description,
        COUNT(zip_code) AS Violation_Count,
        RANK() OVER(
            PARTITION BY zip_code
            ORDER BY COUNT(zip_code) DESC
        ) AS RANK
    FROM PPA_2017_Dataset
    GROUP BY zip_code, Violation_Description
)

SELECT 
    *
FROM TempTable
WHERE RANK = 1
AND zip_code != "0"
GROUP BY zip_code
ORDER BY Violation_Count DESC

"""
run_query(ViolationByZipCodeRanked)

,zip_code,Violation_Description,Violation_Count,RANK
0,19107,METER EXPIRED,75173,1
1,19103,METER EXPIRED,72429,1
2,19104,METER EXPIRED,67225,1
3,19147,METER EXPIRED,66447,1
4,19106,METER EXPIRED,54533,1
5,19146,OVER TIME LIMIT,27774,1
6,19130,METER EXPIRED,22938,1
7,19102,STOP PROHIBITED,16727,1
8,19123,OVER TIME LIMIT,15500,1
9,19127,METER EXPIRED,14234,1


In [43]:
FinesPerZipCodeNJ = """
SELECT 
    zip_code, 
    COUNT(anon_ticket_number) AS fines_given,
    
    SUM(fine) AS sum_fines
FROM PPA_2017_Dataset
WHERE zip_code != "0"
AND state = 'NJ'
GROUP BY zip_code 
ORDER BY fines_given DESC
"""
run_query(FinesPerZipCode)

,zip_code,fines_given,sum_fines
0,19103,178096,9719831
1,19107,162393,9086563
2,19147,150897,6437837
3,19104,139237,6323122
4,19106,108665,5777605
5,19146,71698,2869658
6,19130,67078,2793793
7,19102,51456,3026116
8,19148,49532,1915687
9,19123,47373,1997463
